In [1]:
# imports
import os
from tensorflow import nn
from numpy import zeros
from numpy.random import randint
import tensorflow.keras as keras
from keras.utils import to_categorical,plot_model
from tensorflow.keras import layers, regularizers
# end imports

# Configurations : 

In [1]:
##############
# Params
#############

# DATA
PLANES = 31
MOVES = 361
N = 10_000
DIM = 19

# MODEL
FILTERS = 64
KERNEL_SIZE = (3,3)

# TRAIN
NB_EPOCHS = 20
BATCH_SIZE = 128

# 1)- Shape Data (In/Out) 

In [2]:
'''
    -------------------------------------------------------------------------------------------     
        DataHandler : in this class we handle all stuff related to data shaping and/or
                    data to tensors creations this returned data will be helpful for training
                    and exploited by golois.cpp util function as (getBatch and getValidation).
    -------------------------------------------------------------------------------------------
'''
class DataHandler(object):
    
    def __init__(self,n_samples=N,dim=DIM,n_moves=MOVES,n_planes=PLANES) -> None:
        super().__init__()
        self.n_samples = n_samples
        self.dim = dim
        self.n_moves = n_moves
        self.n_planes = n_planes
    
    def get_data(self):
        # Inputs
        input_data = randint(2, size=(self.n_samples, self.dim, self.dim, self.n_planes)).astype ('float32')

        # Outputs 
        policy = to_categorical(randint(self.n_moves, size=(self.n_samples,)))
        value =  randint(2, size=(self.n_samples,)).astype ('float32')

        # For Get_Batch & Get_Validation
        end = randint(2, size=(self.n_samples, self.dim, self.dim, 2)).astype ('float32')
        groups = zeros((self.n_samples, self.dim, self.dim, 1)).astype ('float32')

        return input_data , policy , value , end , groups

# 2)- DeepGO Model : 

In [3]:
'''
    -------------------------------------------------------------------------------------------     
        DGM (DeepGoModel) : in this class we handle all stuff related to the deep neural model
                            who will represent our GO player all versions with different 
                            architechtures will inheritat this basic methods and added to them
                            their new specific blocks or methods.
    -------------------------------------------------------------------------------------------
'''
class DGM(object):
    
    def __init__(self,version=0,dim=DIM,n_moves=MOVES,n_planes=PLANES,n_filters=FILTERS,
                kernel_size=KERNEL_SIZE,l2_reg=0.0001,dropout=0.2,n_res_blocks=6) -> None:
        super().__init__()
        self.version = version
        self.dim = dim
        self.n_moves = n_moves
        self.n_planes = n_planes
        self.n_filters = n_filters
        self.kernel = kernel_size
        self.l2_reg = regularizers.l2(l2_reg)
        self.dropout = dropout
        self.n_res_blocks = n_res_blocks
    
    def __str__(self) -> str:
        return f'DGMV{self.version}'

    def plot_model(self,save_path='model_imgs'):
        
        if not self.model:
            print(f' You should build the model first !')
            return
        to_file = os.path.join(os.getcwd,save_path,f'{str(self)}.png')

        plot_model(self.model,to_file=to_file,show_shapes=True)
        
    def build_model(self):
        # Input Block
        inp = keras.Input(shape=(self.dim, self.dim, self.n_planes), name='board')
        x = self.input_block(inp)

        # Residual Blocks 
        for _ in range(self.n_res_blocks):
            x = self.residual_block(x)

        # Outputs blocks
        policy_head = self.output_policy_block(x)
        value_head = self.output_value_block(x)

        # Build model 
        self.model = keras.Model(inputs=inp, outputs=[policy_head, value_head])
        return self.model
    
    def input_block(self,inp,kernel_resize=2,pad='same'):
        # CONV2D + BN + activation 
        x = layers.Conv2D(self.n_filters, self.kernel, padding=pad)(inp)
        x = layers.BatchNormalization()(x)
        x = self.activation(x)

        if not kernel_resize:
            return x
        
        # CONV2D (resize) + BN + activation
        new_kernel = tuple(map(lambda x: x+kernel_resize,self.kernel))
        x1 = layers.Conv2D(self.n_filters, new_kernel, padding=pad)(inp)
        x1 = layers.BatchNormalization()(x1)
        x1 = self.activation(x1)
        x = layers.add([x, x1])
        
        return x

    def output_policy_block(self,x):
        policy_head = layers.Conv2D(1, 1, padding='same', use_bias=False, kernel_regularizer=self.l2_reg)(x)
        policy_head = self.activation(policy_head)
        policy_head = layers.BatchNormalization()(policy_head)
        policy_head = layers.Flatten()(policy_head)
        policy_head = layers.Activation('softmax', name='policy')(policy_head)
        return policy_head
        
    def output_value_block(self,x):
        value_head = layers.GlobalAveragePooling2D()(x)
        value_head = layers.Dense(self.n_filters, kernel_regularizer=self.l2_reg)(value_head)
        value_head = self.activation(value_head)
        value_head = layers.BatchNormalization()(value_head)
        value_head = layers.Dropout(self.dropout)(value_head)
        value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=self.l2_reg)(value_head)
        return value_head
    
    def sub_residual_block(self,x,ratio=4):
        x1 = layers.Dropout(self.dropout)(x)
        x1 = layers.GlobalAveragePooling2D()(x1)
        x1 = layers.Dense(self.n_filters//ratio, activation='relu')(x1)
        x1 = layers.Dense(self.n_filters, activation='sigmoid')(x1)
        return layers.Multiply()([x, x1])

    def residual_block(self,x,pad='same'):
        x1 = layers.Conv2D(self.n_filters, self.kernel, padding=pad)(x)
        x1 = layers.BatchNormalization()(x1)
        x1 = self.activation(x1)

        x1 = layers.Conv2D(self.n_filters, self.kernel, padding=pad)(x1)
        x1 = layers.BatchNormalization()(x1)

        x1 = self.se_block(x1)

        x = layers.add([x1, x])
        x = self.activation(x)
        x = layers.BatchNormalization()(x)

    def activation(self,x):
        return nn.leaky_relu(x,alpha=0.3)

In [4]:
dgm_v0 = DGM() 
dgm_v0.plot_model()

NameError: name 'regularizers' is not defined